# <center>Композиции алгоритмов

Будем решать задачу кредитного скоринга.

#### Данные по кредитному скорингу представлены следующим образом:

##### Прогнозируемая  переменная
* SeriousDlqin2yrs	– наличие длительных просрочек выплат платежей за 2 года.

##### Независимые признаки
* age – возраст заёмщика (число полных лет);
* NumberOfTime30-59DaysPastDueNotWorse	– количество раз, когда заёмщик имел просрочку выплаты других кредитов 30-59 дней в течение последних двух лет;
* NumberOfTime60-89DaysPastDueNotWorse – количество раз, когда заёмщик имел просрочку выплаты других кредитов 60-89 дней в течение последних двух лет;
* NumberOfTimes90DaysLate – количество раз, когда заёмщик имел просрочку выплаты других кредитов более 90 дней;
* DebtRatio – ежемесячные отчисления на задолжености (кредиты, алименты и т.д.) / совокупный месячный доход;
* MonthlyIncome	– месячный доход в долларах;
* NumberOfDependents – число человек в семье кредитозаёмщика.

In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split

import matplotlib.pyplot as plt
# %matplotlib inline

In [2]:
data = pd.read_csv("data/credit_scoring_sample.csv", sep=';')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45063 entries, 0 to 45062
Data columns (total 8 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   SeriousDlqin2yrs                      45063 non-null  int64  
 1   age                                   45063 non-null  int64  
 2   NumberOfTime30-59DaysPastDueNotWorse  45063 non-null  int64  
 3   DebtRatio                             45063 non-null  float64
 4   NumberOfTimes90DaysLate               45063 non-null  int64  
 5   NumberOfTime60-89DaysPastDueNotWorse  45063 non-null  int64  
 6   MonthlyIncome                         36420 non-null  float64
 7   NumberOfDependents                    43946 non-null  float64
dtypes: float64(3), int64(5)
memory usage: 2.8 MB


,SeriousDlqin2yrs,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,MonthlyIncome,NumberOfDependents
0,0,64,0,0.249908,0,0,8158.0,0.0
1,0,58,0,3870.000000,0,0,NaN,0.0
2,0,41,0,0.456127,0,0,6666.0,0.0
3,0,43,0,0.000190,0,0,10500.0,2.0
4,1,49,0,0.271820,0,0,400.0,0.0


Заполните медианой пропуски в данных. Выделите в отдельные переменные `X` и `y` матрицу объекты-признаки и целевую переменную.

In [3]:
data.fillna(data.describe().loc["50%"].to_dict(), inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45063 entries, 0 to 45062
Data columns (total 8 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   SeriousDlqin2yrs                      45063 non-null  int64  
 1   age                                   45063 non-null  int64  
 2   NumberOfTime30-59DaysPastDueNotWorse  45063 non-null  int64  
 3   DebtRatio                             45063 non-null  float64
 4   NumberOfTimes90DaysLate               45063 non-null  int64  
 5   NumberOfTime60-89DaysPastDueNotWorse  45063 non-null  int64  
 6   MonthlyIncome                         45063 non-null  float64
 7   NumberOfDependents                    45063 non-null  float64
dtypes: float64(3), int64(5)
memory usage: 2.8 MB


## 1. Дерево решений

Задайте решающее дерево, пользуясь встроенной функцией `DecisionTreeClassifier` с параметрами `random_state=17` и `class_weight='balanced'`.

In [4]:
tree = DecisionTreeClassifier(random_state=17, class_weight="balanced")

Используйте функцию `GridSearchCV` для выбора оптимального набора гиперпараметров для указанной задачи. В качестве метрики качества возьмите ROC AUC.

In [5]:
max_depth_values = [3, 5, 6, 7, 9]
max_features_values = [4, 5, 6, 7]
tree_params = {'max_depth': max_depth_values,
               'max_features': max_features_values}

Зафиксируйте кросс-валидацию с помощью функции `StratifiedKFold` на 5 разбиений с перемешиванием, `random_state=17`.

In [6]:
X = data.drop("SeriousDlqin2yrs", axis=1).values
y = data["SeriousDlqin2yrs"].values
X.shape, y.shape

((45063, 7), (45063,))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.15, random_state=42)
X_train.shape, X_test.shape

((38303, 7), (6760, 7))

In [8]:
%%time

gs = GridSearchCV(
    estimator=tree,
    param_grid=tree_params,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring="roc_auc"
)

gs.fit(X_train, y_train)

Wall time: 5.58 s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=17),
             param_grid={'max_depth': [3, 5, 6, 7, 9],
                         'max_features': [4, 5, 6, 7]},
             scoring='roc_auc')

In [9]:
gs.best_params_

{'max_depth': 6, 'max_features': 7}

Какое максимальное значение ROC AUC получилось?

In [10]:
print("ROC AUC: {:.4f}".format(gs.best_score_))

ROC AUC: 0.8186


При этом `ROC AUC` на тесте: 

In [11]:
print("ROC AUC: {:.4f}".format(roc_auc_score(gs.predict(X_test), y_test)))

ROC AUC: 0.7166


## 2. Реализация бэггинга

Реализуйте класс `BaggingClassifierCustom`. В качестве базового классификатора может выступать любой классификатор, и в реализации необходимо сохранить эту свободу выбора. По умолчанию будем использовать композицию из 10 решающих деревьев, предсказанные вероятности которых необходимо будет усреднить.

Краткая спецификация: 
 - В классе `BaggingClassifierCustom` наследуйте методы `sklearn.base.BaseEstimator`.
 - В методе `fit` в цикле (`i` от 0 до `n_estimators-1`) фиксируйте seed, равный (`random_state + i`). Это нужно для того, чтобы на каждой итерации seed был новый, при этом все значения можно было бы воспроизвести.
 - Зафиксировав seed, сделайте bootstrap-выборку (т.е. **с замещением**) из множества id объектов.
 - Найдите индексы объектов, которые не попали в обучающую выборку.
 - Обучите базовый классификатор на выборке с нужным подмножеством объектов.
 - В методе `predict_proba` нужен цикл по всем базовым алгоритмам композиции. Для тестовой выборки (аргумент `X` в методе) нужно сделать прогноз вероятностей (`predict_proba`). Метод должен вернуть усреднение прогнозов всех базовых алгоритмов.
 - В методе `oob_score` необходимо получить качество (по умолчанию пусть будет ROC AUC) композиции на тех объектах, которые не попали в bootstrap-выборки.

In [12]:
from tsyplov_ensemble import BaggingClassifierCustom

In [13]:
bagging = BaggingClassifierCustom(
    DecisionTreeClassifier(class_weight="balanced", max_depth=5),
    n_estimators=10,
    
    bootstrap=True,
    max_samples=.8,
    min_samples=.4,
    random_state=17, 
    
    oob_score=True,
    oob_scoring=roc_auc_score,
)

Обучаем модель

In [15]:
%%time

bagging.fit(X_train, y_train)

Wall time: 8.76 s


BaggingClassifierCustom(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                              max_depth=5),
                        max_samples=0.8, min_samples=0.4, oob_score=True,
                        oob_scoring=<function roc_auc_score at 0x0000026F5E6DCAF8>,
                        random_state=17)

Считаем результат на тестовой выборке

In [16]:
print("ROC AUC: {:.2f}".format(roc_auc_score(bagging.predict(X_test), y_test)))

ROC AUC: 0.73


Считаем результат `OOB`

In [17]:
print("Out of bag score: {:.2f}".format(bagging.oob_score_))

Out of bag score: 0.73


Оцените качество полученного классификатора на кросс-валидации с метрикой ROC AUC. В качестве базового алгоритма используйте дерево решений с параметрами из первого задания. Какого качества удалось достичь по сравнению с одним решающим деревом?

In [16]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [17]:
%%time

cv_result = []
for train_index, test_index in kf.split(X, y):
    model = bagging.fit(X[train_index], y[train_index])
    y_pred = model.predict(X[test_index])
    cv_result.append(roc_auc_score(y_pred, y[test_index]))

Wall time: 34.5 s


In [18]:
print(
    ("Результаты CV: [" + "{:.4f}, " * 4 + "{:.4f}]").format(*cv_result),
    "Среднее значение CV: {:4f}".format(sum(cv_result) / len(cv_result)),
    sep="\n"
)

Результаты CV: [0.7402, 0.7303, 0.7337, 0.7352, 0.7271]
Среднее значение CV: 0.733287


Ранее были найдены оптимальные гиперпараметры для одного дерева, но может быть, для ансамбля эти параметры дерева не будут оптимальными. С помощью `GridSearchCV` рассмотрите другие варианты. Какими теперь стали лучшие значения гиперпараметров и чему равен ROC AUC?

In [19]:
%%time

bagging_params = {
    'base_estimator__max_depth': [3, 5, 6, 7, 9],
    "min_features": [3, 5, 7],
    "n_estimators": [5, 10, 15, 20, 25]
    
}

bagging = BaggingClassifierCustom(
    DecisionTreeClassifier(class_weight="balanced", max_depth=5), 
    random_state=17, 
    max_samples=20000,  # сильно ускоряет
    oob_score=False,  # сильно ускоряет
    max_features=X.shape[1]
)


gs_b = GridSearchCV(
    estimator=bagging,
    param_grid=bagging_params,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring=make_scorer(roc_auc_score)
)

gs_b.fit(X, y)

Wall time: 1min 43s


In [20]:
print("ROC AUC: {:.2f}".format(roc_auc_score(gs_b.best_estimator_.predict(X), y)))

ROC AUC: 0.72


## 3. Реализация случайного леса

Теперь реализуйте случайный лес. В качестве базового алгоритма здесь всегда выступает `DecisionTreeClassifier`.

Кроме индексов объектов теперь необходимо на каждом шаге алгоритма запоминать также индексы признаков, которые участвовали в обучении леса.

- Зафиксировав seed, выберите **без замещения** `max_features` признаков, сохраните список выбранных id признаков в `self.feat_ids_by_tree`.
- Обучите дерево с теми же `max_depth`, `max_features` и `random_state`, что и у `RandomForestClassifierCustom` на выборке с нужным подмножеством объектов и признаков.
- В методе `predict_proba` у тестовой выборки нужно взять те признаки, на которых соответсвующее дерево обучалось, и сделать прогноз вероятностей (`predict_proba` уже для дерева). Метод должен вернуть усреднение прогнозов по всем деревьям.

***Так получилось, что я случайно реализовал случайный лес внутри бэггинга. Поэтому здесь мы сразу сравним качество с `sklearn`***

Проведите кросс-валидацию. Какое получилось среднее значение ROC AUC на кросс-валидации? Сравните качество вашей реализации с реализацией `RandomForestClassifier` из `sklearn`. Аналогично предыдущему заданию, подберите гиперпараметры для случайного леса.

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [30]:
%%time

forest_params = {
    "max_depth": [1, 2, 3, 5],
    "max_features": ["sqrt", "log2"],
    "n_estimators": [5, 10, 15, 25]
}

forest = RandomForestClassifier(random_state=42, class_weight="balanced")

gs_rf = GridSearchCV(
    estimator=forest,
    param_grid=forest_params,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring=make_scorer(roc_auc_score)
)

gs_rf.fit(X, y)

Wall time: 25.8 s


In [32]:
print("ROC AUC: {:.2f}".format(roc_auc_score(gs_rf.best_estimator_.predict(X), y)))

ROC AUC: 0.72
